<a href="https://colab.research.google.com/github/koll-ai/control-meme-api/blob/main/control_meme_api_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Install dependencies

!pip install "git+https://github.com/takuma104/diffusers.git@controlnet" # Diffusers in development version
!pip install transformers accelerate safetensors xformers opencv-python
!pip install --pre -U triton
!pip install flask-cors
!npm install -g localtunnel

In [3]:
from diffusers import StableDiffusionControlNetPipeline, EulerAncestralDiscreteScheduler
from diffusers.utils import load_image
import torch
import cv2
import numpy as np
from PIL import Image

from flask import Flask
from flask import request
import base64
from io import BytesIO

import subprocess
import time
from flask_cors import CORS

import requests

In [ ]:
#@title # Load ControlNet pipeline

model = "takuma104/control_sd15_canny" #@param ["takuma104/control_sd15_mlsd", "takuma104/control_sd15_hed", "takuma104/control_sd15_seg", "takuma104/control_sd15_depth", "takuma104/control_sd15_scribble", "takuma104/control_sd15_normal", "takuma104/control_sd15_openpose", "takuma104/control_sd15_canny"]

#Common
euler_scheduler = EulerAncestralDiscreteScheduler.from_config(model, subfolder="scheduler")

#Canny Edge model
pipe = StableDiffusionControlNetPipeline.from_pretrained(model, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = euler_scheduler
pipe.enable_xformers_memory_efficient_attention()


In [24]:
import cv2 as cv

def create_hint(image, hint_type):
  init_image = image.resize((512,512))
  
  if hint_type == 'canny':
    controlnet_hint = Image.fromarray(cv.Canny(np.array(init_image), 100,200))
  
  controlnet_hint.save('last_hint.jpeg')

  return controlnet_hint.convert('RGB')

def generate_controlnet(prompt, hint, num_inference_steps, seed=None, negative_prompt=None, **kwargs):
    """
    Generate a meme variation
    POST params: prompt:str, negative_prompt:str, controlnet_hint_url:str, num_inference_steps:int, seed:int
    """
    
    generator = torch.Generator(device="cuda")

    if seed:
      generator.manual_seed(seed)

    image = pipe(prompt=prompt, 
                negative_prompt=negative_prompt,
                image=hint,
                num_inference_steps=num_inference_steps, 
                generator=generator).images[0]
    
    with open('./last_meme.jpeg', 'w') as f:
      image.save(f, format="JPEG")

    return image

In [ ]:
app = Flask(__name__)
CORS(app)

params = dict()

API_URL = ""

@app.route('/hello/')
def hello():    
    return "hello"

@app.route('/', methods=['POST'])
def index():
    global params

    params = request.get_json()

    meme_url = params['controlnet_hint_url']
    image = Image.open(requests.get(meme_url, stream=True).raw)
    hint = create_hint(image, 'canny')

    last_image = generate_controlnet(hint=hint, **params)

    buffered = BytesIO()
    last_image.save(buffered, format="JPEG")

    return base64.b64encode(buffered.getvalue())


@app.route('/save_last/')
def save_last():
    requests.post(f"{API_URL}/api/meme/{params['uuid']}/variation/",
                  files=dict(
                      file=open('./last.jpeg'),
                      prompt=params["prompt"],
                      nb_steps=params["num_inference_steps"]
                      )
                  )    
    return "ok"

if __name__ == '__main__':
  with open('output.txt', 'r') as f:
      contents = f.read()
      API_URL = contents.split(' ')[-1]
      protocol = API_URL.split(':')[0]
      url = API_URL.split('/')[-1]

      print(API_URL)

      print('\n'*2)
      print(f'Everything is ready! Click on the this link to be redirected to koll.ai. Don\'t close this tab!')
      print(f'https://meme.koll.ai?protocol={protocol}&url={url}')

  app.run()

In [ ]:
meme_url = "https://storage.googleapis.com/control-meme-public/hidethepainharold.jpg"

image = Image.open(requests.get(meme_url, stream=True).raw)
image

# get controlnet hint image
hint = create_hint(image, 'canny')
hint

params = {"prompt": "Macron on the phone, french flags",
          "hint": hint,
          "num_inference_steps": 50
        }

b64 = generate_controlnet(**params)
b64